# Set Up

In [ ]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Get Dataset
with zipfile.ZipFile("/content/drive/MyDrive/ADNI_TRAINING_DATASET.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


Mounted at /content/drive


## Define Config



In [ ]:
IMAGE_SIZE = [200, 200]
CLASS_NAMES = ["AD", "CN", "MCI", "pMCI"]

MODEL_SAVE_PATH = "/content/drive/MyDrive/models"

DATASET_PATH = "/content/dataset/ADNI_TRAINING_DATASET"

## Imports

In [ ]:
import sys
from tensorflow.keras.preprocessing import image_dataset_from_directory
from datetime import datetime
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

# Custom Model

In [ ]:
#------------------------------------COGNINET-------------------------------#

def convolutional_block(filters):
    return tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(
            filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(
            filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.AveragePooling2D()
    ]
    )


def dense_block(units, dropout_rate):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])

def cogni_net():
    model = tf.keras.Sequential([
            tf.keras.Input(shape=(*IMAGE_SIZE, 1)),

            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.AveragePooling2D(),

            convolutional_block(32),
            convolutional_block(64),

            convolutional_block(128),
            tf.keras.layers.Dropout(0.2),

            convolutional_block(256),
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Flatten(),
            dense_block(512, 0.7),
            dense_block(128, 0.5),
            dense_block(64, 0.3),

            tf.keras.layers.Dense(
                4, activation='softmax')
        ])
    return model

def alternative_cogni_net_1():
    model = tf.keras.Sequential([
            tf.keras.Input(shape=(*IMAGE_SIZE, 1)),

            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.AveragePooling2D(),

            convolutional_block(32),
            convolutional_block(64),

            convolutional_block(128),
            tf.keras.layers.Dropout(0.2),

            convolutional_block(256),
            tf.keras.layers.Dropout(0.2),

            convolutional_block(512),
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Flatten(),
            dense_block(128, 0.5),
            dense_block(64, 0.3),

            tf.keras.layers.Dense(
                4, activation='softmax')
        ])
    return model

def alternative_cogni_net_2():
    model = tf.keras.Sequential([
            tf.keras.Input(shape=(*IMAGE_SIZE, 1)),

            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.AveragePooling2D(),

            convolutional_block(32),

            convolutional_block(64),
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Flatten(),
            dense_block(256, 0.7),
            dense_block(128, 0.5),
            dense_block(64, 0.3),

            tf.keras.layers.Dense(
                4, activation='softmax')
        ])
    return model


# Models Using Tensorflow Applications

In [ ]:
def get_full_model(base_model):
    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = tf.keras.layers.Dense(4, activation='softmax')(x)

    return tf.keras.models.Model(base_model.input, x)

# ------------------------------------DenseNet-------------------------------#

def DenseNet201():
    base_model = tf.keras.applications.densenet.DenseNet201(
        include_top=False,
        weights=None,
        input_shape=(*IMAGE_SIZE, 1),
        pooling="avg",
        classes=4,
        classifier_activation='softmax'
    )

    return get_full_model(base_model)

# ------------------------------------Inception V3-------------------------------#

def InceptionV3():
    base_model = tf.keras.applications.inception_v3.InceptionV3(
        include_top=False,
        weights=None,
        input_shape=(*IMAGE_SIZE, 1),
        pooling="avg",
        classes=4,
        classifier_activation='softmax'
    )

    return get_full_model(base_model)

# ------------------------------------ResNet-------------------------------#


def ResNet50():
    base_model = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights=None,
        input_shape=(*IMAGE_SIZE, 1),
        pooling="avg",
        classes=4,
        classifier_activation='softmax'
    )

    return get_full_model(base_model)


# ------------------------------------VGGNet-------------------------------#

def VGGNet19():
    base_model = tf.keras.applications.vgg19.VGG19(
        include_top=False,
        weights=None,
        input_shape=(*IMAGE_SIZE, 1),
        pooling="avg",
        classes=4,
        classifier_activation='softmax'
    )

    return get_full_model(base_model)

# Main Model Trainer

In [ ]:
STRATEGY = tf.distribute.get_strategy()
AUTOTUNE = tf.data.AUTOTUNE

BATCH_SIZE = 16 * STRATEGY.num_replicas_in_sync
EPOCHS = 50

MODEL_NAME = "DenseNet201"


base_dir = f"{MODEL_SAVE_PATH}/{MODEL_NAME}-{datetime.now().strftime('%Y-%m-%d-%H:%M')}"
os.makedirs(base_dir, exist_ok=True)

print("REPLICAS: ", STRATEGY.num_replicas_in_sync)

train_ds = image_dataset_from_directory(
    DATASET_PATH,
    labels="inferred",
    label_mode="categorical",
    image_size=IMAGE_SIZE,
    color_mode="grayscale",
    validation_split=0.2,
    subset="training",
    seed=1337
).prefetch(buffer_size=AUTOTUNE)

validation_ds = image_dataset_from_directory(
    DATASET_PATH,
    labels="inferred",
    label_mode="categorical",
    image_size=IMAGE_SIZE,
    color_mode="grayscale",
    validation_split=0.2,
    subset="validation",
    seed=1337
).prefetch(buffer_size=AUTOTUNE)

with STRATEGY.scope():
    model = DenseNet201()

model.summary()

model.compile(
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Defining Callbacks
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"{base_dir}/model.h5", monitor='val_loss', save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    patience=5, monitor='val_accuracy', factor=0.6, min_lr=0.0000001)

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS,
    callbacks=[save_best, reduce_lr]
)

REPLICAS:  1
Found 28800 files belonging to 4 classes.
Using 23040 files for training.
Found 28800 files belonging to 4 classes.
Using 5760 files for validation.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 200, 200, 16)      2320      
                                                                 
 average_pooling2d (Average  (None, 100, 100, 16)      0         
 Pooling2D)                                                      
                                                                 
 sequential (Sequential)     (None, 50, 50, 32)        2160      
                                                                 
 sequential_1 (Sequential)   (None, 25, 25, 64)        7392      
                        

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


720/720 [==============================] - 56s 65ms/step - loss: 1.4847 - accuracy: 0.3270 - val_loss: 1.3240 - val_accuracy: 0.3491 - lr: 0.0010
Epoch 2/50
720/720 [==============================] - 50s 69ms/step - loss: 1.2486 - accuracy: 0.4082 - val_loss: 1.1449 - val_accuracy: 0.4967 - lr: 0.0010
Epoch 3/50
720/720 [==============================] - 46s 64ms/step - loss: 1.1441 - accuracy: 0.4880 - val_loss: 1.1849 - val_accuracy: 0.4668 - lr: 0.0010
Epoch 4/50
720/720 [==============================] - 46s 63ms/step - loss: 1.0317 - accuracy: 0.5602 - val_loss: 0.9429 - val_accuracy: 0.5983 - lr: 0.0010
Epoch 5/50
720/720 [==============================] - 46s 64ms/step - loss: 0.9296 - accuracy: 0.6230 - val_loss: 0.8059 - val_accuracy: 0.6778 - lr: 0.0010
Epoch 6/50
720/720 [==============================] - 50s 70ms/step - loss: 0.8343 - accuracy: 0.6706 - val_loss: 0.6794 - val_accuracy: 0.7337 - lr: 0.0010
Epoch 7/50
720/720 [==============================] - 46s 64ms/step -